In [1]:

import autograd.numpy as np
from qoc import grape_schroedinger_discrete
from qoc.standard import (TargetStateInfidelity,
                          ForbidStates,
                          conjugate_transpose,
                          get_annihilation_operator,
                          get_creation_operator,
                          SIGMA_Z,
                          generate_save_file_path,ControlVariation,Adam,LBFGSB)
from qutip import *

from numpy.fft import fft,fftfreq

# Parameters required

In [43]:
#transmon anharmonicity
k_q = -200e-3 * 2*np.pi  
#transmon truncation level
N_q = 5
#duration
total_time = 50
#number of time intervals for AWG
total_time_steps=2*total_time
#target
angle= np.pi
#sampled detuning point. This is for [-10,10]MHZ. Right now, only uniform ρ(δ) is supported.
fluc_para = 2*np.pi*1e-3*np.array([-10,-7,-4,-1,1,4,7,10]) 
#different initial guess parametrized by (a,b,c). The choice of the range here is just for simple illustration.
a_range = np.arange(-1,2)*2
b_range = np.arange(-1,2)*2
c_range = np.arange(-1,2)*2
points = [(a, b, c) for a in a_range for b in b_range for c in c_range]

In [44]:
# No action required. Following functions return target unitary, initial guess. 

In [45]:
# target X_θ rotation in transmon subspace
def Rx(matrix,angle):
    matrix[0,0]=np.cos(angle/2)
    matrix[0,1]=-1j*np.sin(angle/2)
    matrix[1,0]=-1j*np.sin(angle/2)
    matrix[1,1]=np.cos(angle/2)
    return matrix
# initial guess for y-quadrature control
def pulses_der(x, paras,total_time):
    control = -2*np.pi/total_time*(paras[0]-angle/2)*np.sin(2*np.pi/total_time*x)
    for i in range(len(paras)):
        if i==len(paras)-1:
            control += -2*(i+2)*np.pi/total_time*(-paras[i])*np.sin(2*(i+2)*np.pi/total_time*x)
        else:
            control += -2*(i+2)*np.pi/total_time*(paras[i+1]-paras[i])*np.sin(2*(i+2)*np.pi/total_time*x)
    return control/total_time
# initial guess for x-quadrature control
def pulses(x, paras,total_time):
    control = angle/2 + (paras[0]-angle/2)*np.cos(2*np.pi/total_time*x)
    for i in range(len(paras)):
        if i==len(paras)-1:
            control += (-paras[i])*np.cos(2*(i+2)*np.pi/total_time*x)
        else:
            control += (paras[i+1]-paras[i])*np.cos(2*(i+2)*np.pi/total_time*x)
    return control/total_time

In [46]:
# No action required. Pulse constraints functions.

In [47]:
#bandwidth filter. The frequency components out of [low_freq_lim,high_freq_lim] will be throwed out
def pulse_smoother(pulse,low_freq_lim,
                   high_freq_lim,N,total_time):
    times = np.linspace(0, total_time, N+1)
    times=np.delete(times, [len(times) - 1])
    freq = fftfreq(len(times), times[1]-times[0])
    fourier = fft(pulse)
    for i in range(len(freq)):
        if np.abs(freq[i]) > high_freq_lim:
            fourier[i]=0
        if np.abs(freq[i]) < low_freq_lim:
            fourier[i]=0
    filtered_pulse=np.real(np.fft.ifft(fourier))
    return filtered_pulse
def impose_bc(controls):
    controls[0]=pulse_smoother(controls[0],-1,
                   5/total_time,total_time_steps,total_time)
    controls[1]=pulse_smoother(controls[1],-1,
                    10/total_time,total_time_steps,total_time)
    #force initial/ending control amplitudes to be 0
    controls[0][0]=0
    controls[0][-1]=0
    controls[1][0]=0
    controls[1][-1]=0

    return controls

In [50]:
a_q = destroy(N_q)
n_q = a_q.dag() * a_q
x_q = a_q + a_q.dag()

# without -1/2, strange result when change time origin
H0 =  1/2*k_q * a_q.dag()**2 * a_q**2
Hcx=a_q+a_q.dag()
H0=H0.data.toarray()
Hcx=Hcx.data.toarray()
Hcy=-a_q*1j+1j*a_q.dag()
Hcy=Hcy.data.toarray()
#Hamiltonians for two-quadrature controls
H_controls=[Hcx,Hcy]
#target unitary
target_states=np.zeros([N_q,N_q],dtype=complex)

target_states=Rx(target_states,angle)
Tstate_0 = np.zeros(N_q)
Tstate_0[0] = 1
Tstate_0 = target_states@Tstate_0
Tstate_1 = np.zeros(N_q)
Tstate_1[1] = 1
Tstate_1 = target_states@Tstate_1
target_states = np.array([Tstate_0, Tstate_1])
initial_state0 = np.zeros(N_q)
initial_state0[0] = 1
initial_state1 = np.zeros(N_q)
initial_state1[1] = 1
initial_states = np.array([initial_state0,initial_state1])
cost1=TargetStateInfidelity(target_states=target_states, cost_multiplier= 1)
cost = [cost1]
def robust_operator(para):
    return para*n_q.data.toarray()

robustness_set = [fluc_para, robust_operator]
def control_funcx(controls,time,i):
    return controls[i]
control_func = [control_funcx,control_funcx]
times = np.linspace(0, total_time, total_time_steps+1)
times=np.delete(times, [len(times) - 1])
def optimization(paras):

    initial_controlx= pulses(times,paras,total_time)
    initial_controly= -pulses_der(times,paras,total_time)/(2*k_q)
    initial_control=[initial_controlx,initial_controly]
    result = grape_schroedinger_discrete(H0, H_controls, total_time_steps,
                                         cost, total_time,
                                         initial_states, initial_controls = initial_control,control_func= control_func ,log_iteration_step = 1,
                                         iteration_count=1000, gradients_method="AD", expm_method="pade",impose_control_conditions=impose_bc, robust_set = robustness_set)
    return result,paras


In [ ]:
from joblib import Parallel, delayed
from tqdm import tqdm
results = Parallel(n_jobs=50)(delayed(optimization)(point) for point in tqdm(points));
paras=[]
resultss=[]
for i in range(len(results)):
    paras.append(results[i][1])
    resultss.append(results[i][0])
np.save('pi_result_60_paras',[paras])
np.save('pi_result_60_results',[resultss])

100%|████████████████████████████████████████| 27/27 [00:00<00:00, 18459.04it/s]


iter   |   total error  |       cost0      |   grads_l2  
[3.2e-02 7.6e-03 9.4e-04 1.1e-04 2.7e-05 2.4e-03 1.7e-02 5.9e-02]
  0    | 1.18989958e-01 |  5.87471246e-02  |  6.21983294e+00  
[1.8e-02 1.1e-03 1.2e-03 4.1e-03 4.3e-03 3.2e-03 9.7e-03 4.0e-02]
  1    | 8.17938168e-02 |  4.04201682e-02  |  3.34819333e+00  
[1.0e-02 5.8e-04 5.4e-03 9.8e-03 9.0e-03 3.6e-03 3.0e-03 2.5e-02]
  2    | 6.62721979e-02 |  2.49739235e-02  |  9.85716924e-01  
[7.3e-03 3.0e-03 1.1e-02 1.6e-02 1.5e-02 6.2e-03 4.5e-04 1.6e-02]
  3    | 7.53029764e-02 |  1.62833928e-02  |  2.83709217e+00  
[7.1e-03 4.5e-03 1.4e-02 1.9e-02 1.8e-02 8.0e-03 2.0e-04 1.3e-02]
  4    | 8.39447273e-02 |  1.31215018e-02  |  3.83001461e+00  
[7.3e-03 3.7e-03 1.3e-02 1.8e-02 1.7e-02 7.8e-03 2.4e-04 1.3e-02]
  5    | 8.00374274e-02 |  1.29753544e-02  |  3.54055399e+00  
[8.2e-03 1.9e-03 9.4e-03 1.5e-02 1.4e-02 6.4e-03 5.2e-04 1.5e-02]
  6    | 7.06028254e-02 |  1.47286254e-02  |  2.53376408e+00  
[1.0e-02 6.3e-04 6.1e-03 1.1e-02 1.1e-0

iter   |   total error  |       cost0      |   grads_l2  
[3.2e-01 1.7e-01 6.2e-02 3.9e-03 4.2e-03 6.2e-02 1.7e-01 3.1e-01]
  0    | 1.10911045e+00 |  3.12662204e-01  |  7.27258028e+00  
[2.9e-01 1.6e-01 5.2e-02 2.4e-03 6.3e-03 6.6e-02 1.7e-01 3.1e-01]
  1    | 1.04965722e+00 |  3.05439253e-01  |  6.87336324e+00  
[2.9e-01 1.6e-01 5.5e-02 4.1e-03 4.5e-03 5.6e-02 1.5e-01 2.8e-01]
  2    | 9.94351918e-01 |  2.76097435e-01  |  6.56228603e+00  
[2.8e-01 1.5e-01 5.6e-02 5.1e-03 4.0e-03 5.0e-02 1.4e-01 2.5e-01]
  3    | 9.43796760e-01 |  2.54173576e-01  |  6.36157462e+00  
[2.6e-01 1.4e-01 5.1e-02 4.6e-03 4.8e-03 5.0e-02 1.4e-01 2.4e-01]
  4    | 8.95347856e-01 |  2.43119155e-01  |  6.17459111e+00  
[2.4e-01 1.3e-01 4.5e-02 4.0e-03 5.9e-03 5.1e-02 1.3e-01 2.3e-01]
  5    | 8.49628968e-01 |  2.34741621e-01  |  6.03208422e+00  
[2.3e-01 1.2e-01 4.2e-02 3.9e-03 6.1e-03 4.9e-02 1.3e-01 2.2e-01]
  6    | 8.05689361e-01 |  2.21914484e-01  |  5.90791511e+00  
[2.3e-01 1.2e-01 4.2e-02 4.2e-03 5.5e-0

iter   |   total error  |       cost0      |   grads_l2  
[2.9e-01 1.5e-01 5.0e-02 3.2e-03 3.1e-03 5.0e-02 1.5e-01 3.0e-01]
  0    | 1.00415472e+00 |  3.03606053e-01  |  7.93914025e+00  
[2.5e-01 1.2e-01 3.5e-02 2.5e-03 9.5e-03 6.3e-02 1.7e-01 3.1e-01]
  1    | 9.61436871e-01 |  3.12036105e-01  |  6.24607465e+00  
[2.5e-01 1.3e-01 4.2e-02 6.0e-03 8.9e-03 5.4e-02 1.5e-01 2.9e-01]
  2    | 9.21812374e-01 |  2.85024101e-01  |  5.47372763e+00  
[2.6e-01 1.3e-01 4.9e-02 9.7e-03 8.2e-03 4.4e-02 1.3e-01 2.6e-01]
  3    | 8.88620091e-01 |  2.58081085e-01  |  5.21855215e+00  
[2.6e-01 1.3e-01 5.2e-02 1.2e-02 8.6e-03 3.9e-02 1.2e-01 2.4e-01]
  4    | 8.57547930e-01 |  2.40331730e-01  |  5.16071120e+00  
[2.4e-01 1.3e-01 4.9e-02 1.2e-02 9.5e-03 3.9e-02 1.1e-01 2.3e-01]
  5    | 8.24357680e-01 |  2.32095154e-01  |  5.05439011e+00  
[2.3e-01 1.1e-01 4.3e-02 1.1e-02 1.1e-02 4.1e-02 1.1e-01 2.3e-01]
  6    | 7.90647907e-01 |  2.29054497e-01  |  4.92423508e+00  
[2.1e-01 1.0e-01 3.7e-02 9.9e-03 1.1e-0

iter   |   total error  |       cost0      |   grads_l2  
[3.0e-01 1.5e-01 5.3e-02 3.4e-03 3.4e-03 5.3e-02 1.6e-01 3.0e-01]
  0    | 1.02138064e+00 |  3.00831229e-01  |  7.75112687e+00  
[2.6e-01 1.3e-01 4.1e-02 2.0e-03 7.0e-03 6.0e-02 1.6e-01 3.0e-01]
  1    | 9.65557102e-01 |  2.98338068e-01  |  6.86543075e+00  
[2.6e-01 1.4e-01 4.7e-02 4.7e-03 5.6e-03 5.0e-02 1.4e-01 2.7e-01]
  2    | 9.14772637e-01 |  2.67282616e-01  |  6.22481317e+00  
[2.6e-01 1.4e-01 5.1e-02 7.4e-03 5.3e-03 4.2e-02 1.2e-01 2.4e-01]
  3    | 8.71539042e-01 |  2.40959948e-01  |  5.92990845e+00  
[2.5e-01 1.3e-01 4.8e-02 8.2e-03 6.5e-03 4.1e-02 1.2e-01 2.3e-01]
  4    | 8.29802504e-01 |  2.27456403e-01  |  5.71984456e+00  
[2.3e-01 1.2e-01 4.3e-02 7.8e-03 8.3e-03 4.3e-02 1.2e-01 2.2e-01]
  5    | 7.89368291e-01 |  2.20725155e-01  |  5.55169497e+00  
[2.1e-01 1.1e-01 3.7e-02 7.4e-03 9.7e-03 4.5e-02 1.1e-01 2.1e-01]
  6    | 7.51187056e-01 |  2.14273095e-01  |  5.45404956e+00  
[2.0e-01 1.0e-01 3.5e-02 7.3e-03 1.0e-0

...


 443   | 1.02486526e-02 |  2.85388605e-03  |  1.20950256e-01  
[3.2e-03 7.2e-05 6.0e-04 1.5e-03 1.4e-03 5.1e-04 8.7e-05 2.8e-03]
 444   | 1.02136707e-02 |  2.84410103e-03  |  1.20719387e-01  
[3.2e-03 7.2e-05 6.0e-04 1.5e-03 1.4e-03 5.1e-04 8.7e-05 2.8e-03]
 445   | 1.01787691e-02 |  2.83433553e-03  |  1.20488567e-01  
[3.2e-03 7.2e-05 6.0e-04 1.5e-03 1.4e-03 5.0e-04 8.7e-05 2.8e-03]
 446   | 1.01439481e-02 |  2.82458976e-03  |  1.20257806e-01  
[3.2e-03 7.1e-05 6.0e-04 1.5e-03 1.4e-03 5.0e-04 8.7e-05 2.8e-03]
 447   | 1.01092082e-02 |  2.81486387e-03  |  1.20027111e-01  
[3.2e-03 7.1e-05 5.9e-04 1.5e-03 1.4e-03 5.0e-04 8.7e-05 2.8e-03]
 448   | 1.00745495e-02 |  2.80515806e-03  |  1.19796494e-01  
[3.2e-03 7.1e-05 5.9e-04 1.4e-03 1.4e-03 5.0e-04 8.6e-05 2.8e-03]
 449   | 1.00399725e-02 |  2.79547251e-03  |  1.19565961e-01  
[3.1e-03 7.1e-05 5.9e-04 1.4e-03 1.4e-03 5.0e-04 8.6e-05 2.8e-03]
 450   | 1.00054775e-02 |  2.78580737e-03  |  1.19335524e-01  
[3.1e-03 7.1e-05 5.9e-04 1.4e-03 


 443   | 5.34535196e-03 |  1.52057454e-03  |  1.68832157e-01  
[1.7e-03 9.2e-05 3.0e-04 7.2e-04 7.0e-04 2.7e-04 6.7e-05 1.5e-03]
 444   | 5.31756222e-03 |  1.51213407e-03  |  1.68544051e-01  
[1.6e-03 9.2e-05 3.0e-04 7.1e-04 7.0e-04 2.7e-04 6.7e-05 1.5e-03]
 445   | 5.28994098e-03 |  1.50374457e-03  |  1.68255193e-01  
[1.6e-03 9.1e-05 3.0e-04 7.1e-04 7.0e-04 2.6e-04 6.6e-05 1.5e-03]
 446   | 5.26248703e-03 |  1.49540572e-03  |  1.67965608e-01  
[1.6e-03 9.1e-05 3.0e-04 7.1e-04 6.9e-04 2.6e-04 6.6e-05 1.5e-03]
 447   | 5.23519918e-03 |  1.48711718e-03  |  1.67675320e-01  
[1.6e-03 9.0e-05 3.0e-04 7.0e-04 6.9e-04 2.6e-04 6.6e-05 1.5e-03]
 448   | 5.20807626e-03 |  1.47887862e-03  |  1.67384353e-01  
[1.6e-03 9.0e-05 2.9e-04 7.0e-04 6.8e-04 2.6e-04 6.6e-05 1.5e-03]
 449   | 5.18111709e-03 |  1.47068973e-03  |  1.67092730e-01  
[1.6e-03 9.0e-05 2.9e-04 6.9e-04 6.8e-04 2.6e-04 6.6e-05 1.5e-03]
 450   | 5.15432053e-03 |  1.46255017e-03  |  1.66800473e-01  
[1.6e-03 8.9e-05 2.9e-04 6.9e-04 


 443   | 6.51481097e-03 |  2.09349832e-03  |  1.18590462e-01  
[1.7e-03 8.3e-05 3.2e-04 8.9e-04 9.4e-04 4.3e-04 5.8e-05 2.1e-03]
 444   | 6.47872724e-03 |  2.08227311e-03  |  1.18267296e-01  
[1.7e-03 8.3e-05 3.2e-04 8.8e-04 9.4e-04 4.3e-04 5.8e-05 2.1e-03]
 445   | 6.44283601e-03 |  2.07110379e-03  |  1.17943674e-01  
[1.7e-03 8.2e-05 3.2e-04 8.8e-04 9.3e-04 4.2e-04 5.7e-05 2.1e-03]
 446   | 6.40713636e-03 |  2.05999017e-03  |  1.17619620e-01  
[1.6e-03 8.2e-05 3.2e-04 8.7e-04 9.3e-04 4.2e-04 5.7e-05 2.0e-03]
 447   | 6.37162736e-03 |  2.04893207e-03  |  1.17295154e-01  
[1.6e-03 8.2e-05 3.2e-04 8.7e-04 9.2e-04 4.2e-04 5.7e-05 2.0e-03]
 448   | 6.33630811e-03 |  2.03792931e-03  |  1.16970298e-01  
[1.6e-03 8.1e-05 3.1e-04 8.6e-04 9.2e-04 4.2e-04 5.7e-05 2.0e-03]
 449   | 6.30117771e-03 |  2.02698170e-03  |  1.16645074e-01  
[1.6e-03 8.1e-05 3.1e-04 8.6e-04 9.1e-04 4.2e-04 5.6e-05 2.0e-03]
 450   | 6.26623524e-03 |  2.01608905e-03  |  1.16319503e-01  
[1.6e-03 8.1e-05 3.1e-04 8.5e-04 


 443   | 9.99367470e-03 |  3.34693247e-03  |  3.61788354e-01  
[2.9e-03 4.4e-04 5.8e-04 1.1e-03 1.0e-03 3.8e-04 2.0e-04 3.3e-03]
 444   | 9.97718322e-03 |  3.33986530e-03  |  3.60761922e-01  
[2.9e-03 4.4e-04 5.8e-04 1.1e-03 1.0e-03 3.8e-04 2.0e-04 3.3e-03]
 445   | 9.96067152e-03 |  3.33277694e-03  |  3.59719595e-01  
[2.9e-03 4.4e-04 5.8e-04 1.1e-03 1.0e-03 3.8e-04 2.0e-04 3.3e-03]
 446   | 9.94414098e-03 |  3.32566856e-03  |  3.58661950e-01  
[2.9e-03 4.4e-04 5.7e-04 1.1e-03 1.0e-03 3.8e-04 2.0e-04 3.3e-03]
 447   | 9.92759296e-03 |  3.31854132e-03  |  3.57589569e-01  
[2.9e-03 4.4e-04 5.7e-04 1.1e-03 1.0e-03 3.8e-04 2.0e-04 3.3e-03]
 448   | 9.91102879e-03 |  3.31139639e-03  |  3.56503038e-01  
[2.9e-03 4.4e-04 5.7e-04 1.1e-03 1.0e-03 3.8e-04 2.0e-04 3.3e-03]
 449   | 9.89444976e-03 |  3.30423489e-03  |  3.55402947e-01  
[2.9e-03 4.4e-04 5.6e-04 1.1e-03 1.0e-03 3.8e-04 1.9e-04 3.3e-03]
 450   | 9.87785713e-03 |  3.29705796e-03  |  3.54289886e-01  
[2.9e-03 4.4e-04 5.6e-04 1.1e-03 


 443   | 8.51885223e-04 |  2.09713921e-04  |  2.48866816e-02  
[2.2e-04 3.7e-05 8.8e-05 8.7e-05 8.2e-05 7.8e-05 4.1e-05 2.1e-04]
 444   | 8.44805920e-04 |  2.07572211e-04  |  2.47799211e-02  
[2.2e-04 3.7e-05 8.8e-05 8.6e-05 8.1e-05 7.8e-05 4.1e-05 2.1e-04]
 445   | 8.37813232e-04 |  2.05457681e-04  |  2.46740716e-02  
[2.2e-04 3.7e-05 8.7e-05 8.5e-05 8.0e-05 7.8e-05 4.1e-05 2.0e-04]
 446   | 8.30906205e-04 |  2.03370028e-04  |  2.45691273e-02  
[2.2e-04 3.7e-05 8.7e-05 8.4e-05 7.9e-05 7.7e-05 4.1e-05 2.0e-04]
 447   | 8.24083891e-04 |  2.01308953e-04  |  2.44650822e-02  
[2.2e-04 3.7e-05 8.7e-05 8.3e-05 7.8e-05 7.7e-05 4.1e-05 2.0e-04]
 448   | 8.17345352e-04 |  1.99274159e-04  |  2.43619302e-02  
[2.1e-04 3.7e-05 8.6e-05 8.2e-05 7.7e-05 7.7e-05 4.1e-05 2.0e-04]
 449   | 8.10689660e-04 |  1.97265351e-04  |  2.42596656e-02  
[2.1e-04 3.7e-05 8.6e-05 8.1e-05 7.6e-05 7.6e-05 4.0e-05 2.0e-04]
 450   | 8.04115892e-04 |  1.95282239e-04  |  2.41582822e-02  
[2.1e-04 3.7e-05 8.5e-05 8.0e-05 